# **1-Importação da Base de Dados**


Estamos utilizando uma base pública disponibilizada pelo OpenDataSUS,  Ela reúne notificações de Síndrome Respiratória Aguda Grave (SRAG)

📌 Fonte oficial:
[(https://opendatasus.saude.gov.br/dataset/srag-2021-a-2024)](https://opendatasus.saude.gov.br/dataset/srag-2021-a-2024)

In [ ]:
import pandas as pd
df = pd.read_csv('../raw/INFLUD24-02-06-2025.csv', sep=';', encoding='utf-8', engine='python')
df

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,VG_REINF,REINF,FAB_ADIC,LOT_RE_BI,FAB_RE_BI,DOSE_ADIC,DOS_RE_BI,LOTE_ADIC,TABAG,CASO_SRAG
0,30/01/2024,5,12/01/2024,2,SP,GVE I CAPITAL,1331.0,SAO PAULO,355030,HOSPITAL BOSQUE DA SAUDE,...,NaN,2,NaN,2F3015A,103 - COVID-19 PFIZER - COMIRNATY BIVALENTE,NaN,26/05/2023,NaN,NaN,NaN
1,06/02/2024,6,30/01/2024,5,SC,XANXERE,1556.0,XANXERE,421950,HOSPITAL REGIONAL SAO PAULO ASSEC,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,06/03/2024,10,06/03/2024,10,RS,002 CRS,1607.0,PORTO ALEGRE,431490,HOSPITAL NOSSA SENHORA DA CONCEICAO SA,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,10/03/2024,11,09/03/2024,10,GO,CENTRAL,1779.0,GOIANIA,520870,IRG HOSPITAL,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,01/03/2024,9,24/02/2024,8,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,UPA CAJURU,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268640,25/11/2024,48,20/11/2024,47,SP,GVE VII SANTO ANDRE,1332.0,SAO CAETANO DO SUL,354880,HOSPITAL SAO LUIZ SAO CAETANO,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
268641,21/12/2024,51,16/12/2024,51,RS,017 CRS,1604.0,IJUI,431020,HOSPITAL DE CLINICAS IJUI,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
268642,26/12/2024,52,23/12/2024,52,SP,GVE XXVII SAO JOSE DOS CAMPOS,1351.0,SAO JOSE DOS CAMPOS,354990,HOSPITAL VIVALLE,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
268643,21/11/2024,47,20/11/2024,47,SE,REGIONAL ARACAJU,2056.0,ARACAJU,280030,HOSPITAL SAO LUCAS,...,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


# 2. Seleção das Colunas Relevantes

*   Selecionamos, entre as 191 colunas disponíveis no conjunto original, apenas aquelas essenciais para a análise
*   Reduzimos o conjunto para apenas 17 colunas de interesse, o que torna o processamento mais ágil e direcionado ao objetivo do trabalho.



In [29]:
colunas_necessarias = [
    # Sintomas principais investigados na triagem
    'FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO',
    'DIARREIA', 'VOMITO', 'DOR_ABD', 'FADIGA', 'PERD_OLFT', 'PERD_PALA',

    # Fator de risco (se o paciente declarou possuir algum)
    'FATOR_RISC',

    # Dados relevantes
    'NU_IDADE_N',
    'VACINA_COV',
    'VACINA',

    # Classificação final da doença (campo-alvo)
    'CLASSI_FIN',
    'SG_UF',
]

df = df[colunas_necessarias].copy()


# 3. Remoção de Registros Incompletos (Tratamento de Nulos)

eliminamos todas as linhas com valores ausentes (NaN) em qualquer uma das colunas selecionadas.

In [30]:
df_limpo = df.dropna(subset=colunas_necessarias)
print(f"Número de linhas: {df_limpo.shape[0]}")
print(f"Número de colunas: {df_limpo.shape[1]}")

Número de linhas: 151214
Número de colunas: 18


# 4. Conversão para Binário
De acordo com o Dicionário de Dados SRAG (Fonte: Ministério da Saúde – OpenDataSUS), as variáveis categóricas utilizadas para sintomas e fatores de risco possuem a seguinte codificação:

* 1: Sim
* 2: Não
* 9: Não sabe

Convertemos os valores padronizados para formato binário, onde:

**1** representa presença e o **2 e 9** representa ausência



In [31]:
colunas_preservar = ['NU_IDADE_N', 'CLASSI_FIN']
for col in df.columns:
    if col not in colunas_preservar:
        df_limpo[col] = df_limpo[col].apply(lambda x: 1 if x == 1 else 0)

<ipython-input-31-1520340300>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpo[col] = df_limpo[col].apply(lambda x: 1 if x == 1 else 0)


# 6. Mapeamento do Diagnóstico Final

Segundo as infos do dicionário de dados do banco SRAG, a cerca da  coluna CLASSI_FIN .Para nosso estudo, os valores mais relevantes são:

5 → COVID-19 confirmado

4 → Influenza confirmada

Criamos uma nova coluna "diagnóstico", classificando os registros como covid, influenza ou outro (para os demais casos).

In [32]:
mapeamento = {5: 'covid', 4: 'influenza'}
df_limpo['diagnóstico'] = df_limpo['CLASSI_FIN'].map(mapeamento).fillna('outro')


<ipython-input-32-2742758922>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpo['diagnóstico'] = df_limpo['CLASSI_FIN'].map(mapeamento).fillna('outro')


In [33]:
contagem = df_limpo['diagnóstico'].value_counts()
percentual = df_limpo['diagnóstico'].value_counts(normalize=True) * 100

for classe in contagem.index:
    print(f"{classe}: {contagem[classe]} casos ({percentual[classe]:.2f}%)")

influenza: 81083 casos (53.62%)
outro: 52201 casos (34.52%)
covid: 17930 casos (11.86%)


# 7. Filtragem por Grupo Prioritário e Distribuição dos Casos
Para focar em um grupo de maior risco, filtramos os dados para incluir apenas indivíduos **com 60 anos ou mais (NU_IDADE_N >= 60).**
Em seguida, analisamos a distribuição de diagnósticos (covid, influenza, outro) dentro desse grupo

In [34]:
df_prioritarios = df_limpo[df_limpo['NU_IDADE_N'] >= 60].copy()
print(f"Número de linhas: {df_prioritarios.shape[0]}")



Número de linhas: 46616


In [35]:
contagem = df_prioritarios['diagnóstico'].value_counts()
percentual = df_prioritarios['diagnóstico'].value_counts(normalize=True) * 100

for classe in contagem.index:
    print(f"{classe}: {contagem[classe]} casos ({percentual[classe]:.2f}%)")

influenza: 26857 casos (57.61%)
covid: 10982 casos (23.56%)
outro: 8777 casos (18.83%)


In [36]:
df_prioritarios

,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,FATOR_RISC,NU_IDADE_N,VACINA_COV,VACINA,CLASSI_FIN,SG_UF,diagnóstico
1,0,0,0,1,1,1,1,1,1,0,0,0,1,71,1,0,4.0,0,influenza
2,0,1,0,1,0,1,0,0,0,1,0,0,1,81,1,0,4.0,0,influenza
5,0,0,0,0,1,1,1,1,1,1,0,0,0,75,0,0,4.0,0,influenza
12,1,0,1,0,0,0,0,0,0,1,0,0,1,91,1,0,5.0,0,covid
15,0,1,0,1,0,1,0,0,0,0,0,0,1,90,1,0,5.0,0,covid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268633,0,1,0,0,0,0,0,0,0,0,0,0,0,85,1,1,4.0,0,influenza
268634,0,1,0,1,0,1,0,0,0,0,0,0,1,82,1,1,4.0,0,influenza
268637,0,0,0,1,1,1,0,0,0,0,0,0,1,84,1,0,4.0,0,influenza
268638,0,0,0,1,0,1,0,0,0,0,0,0,1,80,1,0,4.0,0,influenza


# 8. Remoção de Perfis Sintomáticos Duplicados

Após filtrar os registros por grupo prioritário (idosos ≥ 60 anos) e padronizar os dados, aplicamos uma deduplicação baseada apenas nas colunas de sintomas e diagnóstico final. Isso significa que:

* Pacientes com exatamente o mesmo conjunto de sintomas e mesmo diagnóstico (covid, influenza ou outro) foram considerados redundantes.

* Apenas um representante de cada combinação única foi mantido no conjunto final.

**Essa decisão não compromete nossa POC, pois o foco inicial está em identificar padrões de sintomas respiratórios entre idosos**

In [50]:
colunas_unicas = [
    'FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO',
    'DIARREIA', 'VOMITO', 'DOR_ABD', 'FADIGA', 'PERD_OLFT', 'PERD_PALA',
    'diagnóstico',
]

df_final = df_prioritarios.drop_duplicates(
    subset=colunas_unicas
)


In [54]:
contagem = df_final['diagnóstico'].value_counts()
percentual = df_final['diagnóstico'].value_counts(normalize=True) * 100

for classe in contagem.index:
    print(f"{classe}: {contagem[classe]} casos ({percentual[classe]:.2f}%)")

influenza: 1007 casos (40.62%)
covid: 798 casos (32.19%)
outro: 674 casos (27.19%)


# 9. Salvando o Conjunto Final de Dados

In [ ]:
df_final.to_csv('../processed/INFLUD-IDOSOS_TRATADO.csv', index=False)